In [1]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/TrabalhoPL/gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create gold database
spark.sql(
    """
    DROP DATABASE IF EXISTS Projeto_gold CASCADE
    """
)
spark.sql(
    """
    create database Projeto_gold location 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db'
    """
)


DataFrame[]

In [2]:
from pyspark.sql.functions import substring, avg, sum

# read air_quality from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Ar"

air_quality = spark\
             .read\
             .load(hdfs_path)

air_quality.show()
air_quality.printSchema()

+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-----------+----------+----------+----------+-------------+----+
|Unique_ID|Indicator_ID|                Name|       Measure|Measure_Info|Geo_Type_Name|Geo_Join_ID|      Geo_Place_Name|Time_Period|Start_Date|Data_Value|  End_Date|   Localidade| Ano|
+---------+------------+--------------------+--------------+------------+-------------+-----------+--------------------+-----------+----------+----------+----------+-------------+----+
|   179718|         642|Boiler Emissions-...|Number per km2|      number|        UHF42|        504|South Beach - Tot...|       2015|2015-01-01|       2.0|2015-12-31|Staten Island|2015|
|   179719|         642|Boiler Emissions-...|Number per km2|      number|        UHF42|        503|         Willowbrook|       2015|2015-01-01|       2.1|2015-12-31|Staten Island|2015|
|   179720|         642|Boiler Emissions-...|Number per km2|      number|  

In [10]:
from pyspark.sql.functions import count
gold_air_quality = air_quality \
    .groupBy("Ano", "Localidade", "Name")\
    .agg(
        sum(air_quality.Data_Value).alias("Soma")
    ) \
   
gold_air_quality.toPandas()


Ano     Localidade                                               Name  \
0    2009      Manhattan                                         Ozone (O3)   
1    2009  Staten Island                          PM2.5-Attributable Deaths   
2    2015       Brooklyn                               Sulfur Dioxide (SO2)   
3    2009      Manhattan  PM2.5-Attributable Asthma Emergency Department...   
4    2015          Bronx  PM2.5-Attributable Respiratory Hospitalization...   
..    ...            ...                                                ...   
535  2008         Queens                             Nitrogen Dioxide (NO2)   
536  2020      Manhattan                    Fine Particulate Matter (PM2.5)   
537  2020      Manhattan                             Nitrogen Dioxide (NO2)   
538  2020       Brooklyn                             Nitrogen Dioxide (NO2)   
539  2020  New York City                    Fine Particulate Matter (PM2.5)   

            Soma  
0     554.539998  
1     261.399998  
2       7.950000  
3    1910.499997  
4     158.799998  
..           ...  
535  1674.790000  
536   400.969999  
537   979.690002  
538  1128.349998  
539   111.790000  

[540 rows x 4 columns]

In [13]:
# create air_quality table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto_gold.Tabela_Ar_3
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Ar_3 (
        Ano INT,
        Localidade VARCHAR(250), 
        Name VARCHAR(500), 
        Soma DOUBLE
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Ar_3/'
    """
)
    

DataFrame[]

In [14]:
# write to delta table
gold_air_quality \
    .write  \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Ar_3/")

In [15]:
# check the results in the table
spark.table("Projeto_gold.Tabela_Ar_3").toPandas()


Ano     Localidade                                               Name  \
0    2009      Manhattan                                         Ozone (O3)   
1    2009  Staten Island                          PM2.5-Attributable Deaths   
2    2015       Brooklyn                               Sulfur Dioxide (SO2)   
3    2009      Manhattan  PM2.5-Attributable Asthma Emergency Department...   
4    2015          Bronx  PM2.5-Attributable Respiratory Hospitalization...   
..    ...            ...                                                ...   
535  2008         Queens                             Nitrogen Dioxide (NO2)   
536  2020      Manhattan                    Fine Particulate Matter (PM2.5)   
537  2020      Manhattan                             Nitrogen Dioxide (NO2)   
538  2020       Brooklyn                             Nitrogen Dioxide (NO2)   
539  2020  New York City                    Fine Particulate Matter (PM2.5)   

            Soma  
0     554.539998  
1     261.399998  
2       7.950000  
3    1910.499997  
4     158.799998  
..           ...  
535  1674.790000  
536   400.969999  
537   979.690002  
538  1128.349998  
539   111.790000  

[540 rows x 4 columns]

In [16]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Ar_3/`
""").show()

++
||
++
++



In [18]:
spark.sql("""
DROP TABLE IF EXISTS Projeto_gold.Tabela_Ar_Presto_3 
""").show()

spark.sql("""
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Ar_Presto_3 (
        Ano INT,
        Localidade VARCHAR(250), 
        Name VARCHAR(500), 
        Soma DOUBLE 
        )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Ar_3/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

